In [11]:
import json
import hashlib
import os
import random
import requests
import time

from datetime import datetime

In [12]:
# Declarando as variaveis
datetime_pattern = '%Y-%m-%d %H:%M:%S'
payload_path = "logs/received/instructions/"
dir_marvel_get = "logs/get_marvel"
dir_marvel_get_error = "logs/error_get_marvel"
post_professor = "logs/post_professor"
instructions_ok = "logs/instructions_ok"
instructions_not_ok = "logs/instructions_not_ok"
url_marvel_base = "http://gateway.marvel.com/v1/public"
post_url_professor = ""
os.makedirs(dir_marvel_get, exist_ok=True)
os.makedirs(dir_marvel_get_error, exist_ok=True)
os.makedirs(post_professor, exist_ok=True)
os.makedirs(instructions_ok, exist_ok=True)
os.makedirs(instructions_not_ok, exist_ok=True)

In [13]:
# Escrever arquivos

def escrever_arquivos(data_, path, id):
    with open(f"{path}/{id}.json", "w") as file:
        file.write(json.dumps(data_))

In [14]:
# Ler os arquivos
def ler_arquivos(path):
    payload_list = []

    file_list = os.listdir(path)

    for file_ in file_list:
        with open(f"{path}/{file_}") as file:
            payload_list.append(json.load(file))
    return payload_list

In [15]:
# Criar o hash md5 para a chamada de api da marvel
def create_hash(timestamp, private_key, public_key):

    str_to_hash = str(timestamp) + str(private_key) + str(public_key)
    hash_md5 = hashlib.md5(str_to_hash.encode())
    return hash_md5.hexdigest()

In [16]:
# Ler o arquivo de credenciais da marvel
with open("api.json") as file:
    credentials = json.load(file)

public_key_ = credentials['marvel']['public']
private_key_ = credentials['marvel']['private']


In [17]:
# Carrega uma lista contendo todas as payloads em conformidade recebidas pelo server
payload_list = ler_arquivos(payload_path)

for payload in payload_list:
    orientacao = "instruction"
    orientacao_recebida = f"{payload['type']}"
    id_ = f"{payload['body']['entity_name']}_{payload['body']['instruction_id']}_{payload['body']['entity_id']}_{datetime.now().strftime('%Y%m%d-%H%M%S-%f')}"

    if orientacao_recebida == orientacao:
        escrever_arquivos(data_=payload, path=instructions_ok, id=id_)
    else:
        print(f"Esse não é uma instrução: {payload}")
        escrever_arquivos(data_=payload, path=instructions_not_ok, id=id_)

Esse não é uma instrução: {'type': 'truction', 'body': {'instruction_id': 463, 'entity_name': 'events', 'entity_id': 277, 'timestamp': '2023-04-10 11:33:35'}}


In [ ]:
# Carrega uma lista contendo todas as payloads em conformidade recebidas pelo server
payload_list = ler_arquivos(instructions_ok)

# Realiza um loop para retornar todos os dados brutos da API da Marvel, para cada requisição encontrada na lista
for payload in payload_list:
    request_url = f"{url_marvel_base}/{payload['body']['entity_name']}/{payload['body']['entity_id']}"
    ts_ = datetime.now().timestamp()

    params_ = {
        "ts": ts_,
        "apikey": public_key_,
        "hash": create_hash(timestamp=ts_, private_key=private_key_, public_key=public_key_)
    }

    response = requests.get(url=request_url, params=params_)
    response_ = response.json()
    id_ = f"{payload['body']['entity_name']}_{payload['body']['instruction_id']}_{payload['body']['entity_id']}_{datetime.now().strftime('%Y%m%d-%H%M%S-%f')}"

    if response.status_code == 200:
        escrever_arquivos(data_=response_, path=dir_marvel_get, id=id_)
    else:
        print(f"Deu erro na chamada da api da Marvel: {payload}")
        data_dict_ = {
            "instruction_id": payload['body']['instruction_id'],
            "entity_name": payload['body']['entity_name'],
            "entity_id": payload['body']['entity_id'],
            "error": response_
        }
        escrever_arquivos(data_=data_dict_, path=dir_marvel_get_error, id=id_)

    time.sleep(0.5)

### A partir daqui eu acabei pegando os dados manuais pois não consegui tempo para continuar, tive alguns compromissos no feriado 